In [1]:
import numpy as np
import h5py
from scipy.ndimage import zoom, laplace, gaussian_filter, rotate
from scipy.interpolate import RegularGridInterpolator as RGI
from scipy.interpolate import splprep, splev
import matplotlib.pyplot as plt
from matplotlib.path import Path
import os
from tqdm import tqdm
import json
import traceback
import pydicom
import pickle
import datetime
import copy
import csv
import pandas as pd
import multiprocessing as mp
import gc
# Test if gdcm works for decompressing JPEG 2000
print("IT IS {}, GDCM IS LOCKED AND LOADED".format(pydicom.pixel_data_handlers.gdcm_handler.is_available()))

# import cv2
# from mpl_toolkits.mplot3d import Axes3D
# import matplotlib
# matplotlib.use('Agg')


IT IS True, GDCM IS LOCKED AND LOADED


In [4]:
PROJECTPATH = '/home/ayeluru/4d-flow-automation/'
DATAPATH = os.path.join(PROJECTPATH,'ge-testing')
ANNOPATH = os.path.join(PROJECTPATH, 'ge-testing')

FLOWPATH = os.path.join(DATAPATH, 'ge-volumetric-cardiac-4dflow.hdf5')
METAPATH = os.path.join(DATAPATH,'ge-cardiac-4dflow-metadata.pkl')

# CENTROIDPATH = os.path.join(ANNOPATH,'flow_2d_cartesian_centroids_19April2023.pkl')

In [5]:
with open(METAPATH, "rb") as metafile:
    allmeta = pickle.load(metafile)
    
# with open(CENTROIDPATH, "rb") as cf:
#     allcentroids = pickle.load(cf)
flowdata = h5py.File(FLOWPATH, 'r')

In [25]:
flowdata.close()

In [7]:
output.close()

In [6]:
OUTPUT = os.path.join(DATAPATH, 'ge-volumetric-cardiac-magdata.hdf5')
# OUTCOORDS = os.path.join(ANNOPATH, 'localizations_rcs_coords_19April2023.pkl')
OUTSCALING = os.path.join(ANNOPATH, 'ge-localizations_rcs_scale_factors_040225.pkl')

try:
    os.remove(OUTPUT)
#     os.remove(OUTCOORDS)
#     os.remove(OUTSCALING)
except:
    pass
output = h5py.File(OUTPUT, 'a')

In [21]:
print(output)

<HDF5 file "volumetric-cardiac-magdata.hdf5" (mode r+)>


In [7]:
def to_rcs(vec, A):
    temp = np.ones((4,1))
    temp[0:3,0] = vec
    return np.squeeze(np.matmul(A,temp)[0:3])


DIMS = [192.,192.,64.]
outcoords = dict()
outscaling = dict()
for k in tqdm(list(flowdata.keys())):
    if not k in output.keys():
        Ainv = allmeta[k]['Ainv_flow']
        origshp = flowdata[k].shape
        rfactor = DIMS[0] / origshp[0]
        cfactor = DIMS[1] / origshp[1]
        sfactor = DIMS[2] / origshp[2]
        factors = [rfactor, cfactor, sfactor]
        outscaling[k] = factors
#         outcoords[k] = dict()
#         # Handle regular centroids first
#         regular_list = ['AV','Ao','PV', 'PA']
#         for s in regular_list:
#             try:
#                 T = len(allcentroids[k+'_'+s])
#                 netc = np.zeros((T, 3))
#                 for t in range(T):
#                     currc = to_rcs(allcentroids[k+'_' + s][t], Ainv)
#                     currc = (currc*factors)
#                     netc[t,:] = currc
#                 outcoords[k][s] = netc
#             except:
#                 traceback.print_exc()
#                 pass
            
        imdata = np.array(flowdata[k][:,:,:,:,0]).astype('int16')
        imout = zoom(imdata, (rfactor, cfactor, sfactor, 1), order=1)
        
        try:
            del output[k]
        except:
            pass

        output.create_dataset(k, data=imout, dtype=np.int16, compression='lzf')
        print('Completed successfully @ {}, shape {}'.format(k, imout.shape))
            
with open(OUTCOORDS, 'wb') as f:
    pickle.dump(outcoords, f)
with open(OUTSCALING, 'wb') as f:
    pickle.dump(outscaling, f)

  6%|██████▍                                                                                               | 1/16 [00:36<09:11, 36.79s/it]

Completed successfully @ Bibipnox, shape (192, 192, 64, 30)



 12%|████████████▊                                                                                         | 2/16 [00:59<06:35, 28.24s/it]

Completed successfully @ Cakimtol, shape (192, 192, 64, 20)



 19%|███████████████████▏                                                                                  | 3/16 [01:35<06:53, 31.84s/it]

...


 94%|██████████████████████████████████████████████████████████████████████████████████████████████▋      | 15/16 [08:50<00:29, 29.62s/it]

Completed successfully @ Quisculjug, shape (192, 192, 64, 20)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [09:23<00:00, 35.24s/it]

Completed successfully @ Tabipa, shape (192, 192, 64, 30)


NameError: name 'OUTCOORDS' is not defined

In [9]:
OUTSCALING = os.path.join(DATAPATH, 'ge-localizations_rcs_scale_factors_040225.pkl')
with open(OUTSCALING, 'wb') as f:
    pickle.dump(outscaling, f)


In [10]:
output.close()

In [35]:
"""
Loop
"""
def to_rcs(vec, A):
    temp = np.ones((4,1))
    temp[0:3,0] = vec
    return np.squeeze(np.matmul(A,temp)[0:3])

    
DIMS = [64.,64.,64.]
for k in tqdm(list(flowdata.keys())):
    try:
        if not k in output.keys():
            Ainv = allmeta[k]['Ainv_flow']
            origshp = flowdata[k].shape
            rfactor = DIMS[0] / origshp[0]
            cfactor = DIMS[1] / origshp[1]
            sfactor = DIMS[2] / origshp[2]
            factors = [rfactor, cfactor, sfactor]
            outscaling[k] = factors
            outcoords[k] = dict()
            # Handle regular centroids first
            regular_list = ['AV','Ao','PV', 'PA']
            for s in regular_list:
                try:
                    T = len(allcentroids[k+'_'+s])
                    netc = np.zeros((T, 3))
                    for t in range(T):
                        currc = to_rcs(allcentroids[k+'_' + s][t], Ainv)
                        currc = (currc*factors)
                        netc[t,:] = currc
                    outcoords[k][s] = netc
                except:
                    traceback.print_exc()
                    pass
            imdata = np.array(flowdata[k][:,:,:,:,0]).astype('int16')
            imout = zoom(imdata, (rfactor, cfactor, sfactor, 1), order=1)
            try:
                del output[k]
            except:
                pass
            output.create_dataset(k, data=imout, dtype=np.int16, compression='lzf')
            print('Completed successfully @ {}, shape {}'.format(k, imout.shape))
    except:
        pass
with open(OUTCOORDS, 'wb') as f:
    pickle.dump(outcoords, f)
with open(OUTSCALING, 'wb') as f:
    pickle.dump(outscaling, f)
# output.close()

  3%|██▍                                                                                   | 1/35 [00:37<21:25, 37.82s/it]

Completed successfully @ Besapol, shape (64, 64, 64, 20)



  6%|████▉                                                                                 | 2/35 [01:16<21:07, 38.41s/it]

Completed successfully @ Deefegi, shape (64, 64, 64, 20)



  9%|███████▎                                                                              | 3/35 [01:46<18:29, 34.68s/it]

...

Completed successfully @ Tibomol, shape (64, 64, 64, 20)



 97%|██████████████████████████████████████████████████████████████████████████████████▌  | 34/35 [17:27<00:29, 29.50s/it]

Completed successfully @ Tiheerien, shape (64, 64, 64, 20)


100%|█████████████████████████████████████████████████████████████████████████████████████| 35/35 [17:58<00:00, 30.80s/it]

Completed successfully @ Urquidash, shape (64, 64, 64, 20)


In [10]:
output.close()

In [41]:
output = h5py.File(OUTPUT, 'r')